# Study TimeDiffs code, algorithm

In [1]:
%matplotlib inline
plt.rc('font', size=12)

## Code copied brom EZPsearch

In [2]:
def FFT_Size(window_size, max_freq):
  """FFT_Size()"""
  """The size of the FFT is twice the product of"""
  """the window size times the maximum frequency."""
  return 2 * int(np.floor(window_size * max_freq))+1 # needed if integer time diff

def TimeDiffs(times, weights, window_size = 524288, max_freq = 64):
  """TimeDiffs()"""
  """Extract the binned series of time differences"""
  """The argument max_freq determines the bin size"""
  """as time_resol = 1 / (2 * max_freq) """
  """This together with window_size fixes the size"""
  """of the returned array of time differences"""
  # FFT sampling time
  time_resol = .5 / max_freq
  # directly bin the time differences
  time_diffs = [0] * FFT_Size(window_size, max_freq)
  for i1 in range(len(times) - 1):
    t1 = times[i1]
    for i2 in range(i1 + 1, len(times)):
      t2 = times[i2]
      # limit the size of the time differences
      if t2 - t1 > window_size:
        break
      # determine the frequency bin in the array
      freq_bin = int(np.floor((t2 - t1) / time_resol))
      # combine the weights appropriately
      time_diffs[freq_bin] += weights[i1] * weights[i2]
  return time_diffs

### Set up a simple data set, window and resolution parameters

In [3]:
times = np.arange(1e4)*100.
weights = np.ones(len(times))
window_size, max_freq = (1e5, 0.1)
fft_size = 2 * int(np.floor(window_size * max_freq))
time_resol = .5 / max_freq

fft_size, time_resol

(20000, 5.0)

### time TimeDiffs

In [4]:
%time ts1=TimeDiffs(times, np.ones(fft_size+1), window_size, max_freq)

CPU times: user 45.5 s, sys: 243 ms, total: 45.8 s
Wall time: 45.9 s


### time using numpy alternative

In [5]:
%%time
td = np.zeros(fft_size+1)
for i1,t1 in enumerate(times):
    b = np.searchsorted(times, ( t1+window_size))
    t2 = times[i1+1:b]
    fb = np.floor((t2-t1)/time_resol).astype(int)
    td[fb] += weights[i1]*weights[i1+1:b]

CPU times: user 572 ms, sys: 2 ms, total: 574 ms
Wall time: 581 ms


### compare
Factor of 100!

In [6]:
np.unique(td-ts1, return_counts=True)

(array([-9000.,     0.]), array([    1, 20000]))

<b>So the first bin is different. Look into that later